In [1]:
import os
import subprocess
from astropy.table import Table
from astropy.io import ascii

In [2]:
# diffDir = '/media/mj1e16/PP AV-TV/properDiff/'
# medianDir = '/media/mj1e16/PP AV-TV/medians/'
# imageDir = '/media/mj1e16/PP AV-TV/keplerCal/'
# sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'

In [3]:
diffDir = '/data/mj1e16/kepler/properDiff/'
medianDir = '/data/mj1e16/kepler/medians/'
imageDir = '/data/mj1e16/kepler/cal/'
sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'

In [17]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType,colour='red',radius='2'):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sexMedian':
            ra = sexTabList[tables]['RA']
            dec = sexTabList[tables]['DEC']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'image\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},'.format(xcoords[x],ycoords[x])+radius+') # color={}\n'.format(colour)
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)

In [24]:
def makeConfig(valList,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE']):
    

    with open(defaultDir+'/default_copy.sex','r') as f:
        data  = f.read()
    
    catLocFinder = 'CATALOG_NAME'
    catname = 'test_test.cat'
    nameLoc = data.find(catLocFinder) + len(catLocFinder)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' '+catname+' ' + data[endLoc:]
    data = newData 
        
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[0]) + len(attributeList[0])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData    
    
    
    confName = 'default_test.sex'
    with open(defaultDir+confName,'w') as f:
        f.write(data)
    

In [35]:
def findObjects(valList,atList,image,index):
    makeConfig(valList,attributeList=atList)
    subprocess.call(['sex',image,'-c','default_test.sex'])
    tabTot =  ascii.read('/home/mj1e16//sextractor/sextractor-master/config/test_test.cat')
    print(len(tabTot))
    makeDS9RegFile([tabTot],'sextest_{}'.format(index),'sex')

In [48]:
atList = ['FILTER_NAME']
#valList = [[],[],[],[],[],[]]

#valList = [[5],[7],[9],[11],[13],[15]] best values -dt 8 dma 10 mexhat mexhat_4.0_9x9.conv background does not matter gauss - 'gauss_5.0_9x9.conv' tophat is bad
#valList = [[6],[7],[8],[9],[10],[11]] # 7 detect thresh is more complete, 8 is more accurate
#valList = [['mexhat_1.5_5x5.conv'],['mexhat_2.0_7x7.conv'],['mexhat_2.5_7x7.conv'],['mexhat_3.0_9x9.conv'],['mexhat_4.0_9x9.conv'],['mexhat_5.0_11x11.conv']]
#valList = [['gauss_1.5_3x3.conv'],['gauss_2.0_3x3.conv'],['gauss_2.0_5x5.conv'],['gauss_2.5_5x5.conv'],['gauss_3.0_5x5.conv'],['gauss_3.0_7x7.conv'],['gauss_4.0_7x7.conv'],['gauss_5.0_9x9.conv']]
#valList = [['tophat_1.5_3x3.conv'],['tophat_2.0_3x3.conv'],['tophat_2.5_3x3.conv'],['tophat_3.0_3x3.conv'],['tophat_4.0_5x5.conv'],['tophat_5.0_5x5.conv']]
valList = [['mexhat_4.0_9x9.conv'],['gauss_5.0_9x9.conv']]
image = 'diff_imNo33_channel44_ccd44.fits'

In [21]:
os.chdir(sexDir)


In [49]:
for x in range(len(valList)):
    findObjects(valList[x],atList,diffDir+image,x)


102
65


In [50]:
command = ['ds9']
for x in range(len(valList)):
    command.extend([diffDir+image,'-regions','sextest_{}.reg'.format(x),'-scale','mode','zscale'])
command.extend(['-frame','lock','image'])
print(command)
subprocess.call(command)

['ds9', '/data/mj1e16/kepler/properDiff/diff_imNo33_channel44_ccd44.fits', '-regions', 'sextest_0.reg', '-scale', 'mode', 'zscale', '/data/mj1e16/kepler/properDiff/diff_imNo33_channel44_ccd44.fits', '-regions', 'sextest_1.reg', '-scale', 'mode', 'zscale', '-frame', 'lock', 'image']


0